## Notebook B: Linking the transport network/s

**TU Delft**<br>
**Author:** Ruth Nelson <br>

An example notebook showing how to link the transport vertices to the network created in Notebook A. 
You will run this notebook for as many times as you have transport networks, in the case of the Cape Town sample data twice, once for the BRT and once for the Railway.

1. Import libraries
2. Import Network A created in Notebook A and transport vertices
3. Cleaning
4. Seperate vertices and edges and concatenate
5. Snkit to link transport vertices with Network A
6. Adjust attributes
7. Saving the files to shp

### 1. Import libraries

In [ ]:
import geopandas as gpd
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import snkit
import os
import glob

In [ ]:
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', None)
#pd.set_option('display.notebook_repr_html', True)

### 2. Import the dataframes

In [ ]:
#import the linked road network to land use and the seperate transport network

In [ ]:
path0 = " "

In [ ]:
os.chdir(path0)
for file in glob.glob("*"):
    print(file)

In [ ]:
edges  = gpd.read_file('edges_a.shp')

In [ ]:
vertices = gpd.read_file('vertices_a.shp')

In [ ]:
#import vertices from transport network

In [ ]:
path1 = " "

In [ ]:
os.chdir(path1)
for file in glob.glob("*"):
    print(file)

In [ ]:
transport_vertices  = pd.read_csv('Myciti_vertices.csv') # load in transport vertices

In [ ]:
transport_vertices

### 3. Cleaning

In [ ]:
vertices

In [ ]:
transport_vertices.head()

In [ ]:
#rename the name column
transport_vertices = transport_vertices.rename(columns = {'name':'id'})

In [ ]:
#adding landuse and vertex_type columns
transport_vertices['land use'] = 'brt_stop' #could be another kind of stop/station, just an example
transport_vertices['vertex_typ'] = 'not_street'

In [ ]:
#ensure that transport vertices have geometry

from shapely.geometry import Point

# Create geometry column from x and y
geometry = [Point(xy) for xy in zip(transport_vertices['x'], transport_vertices['y'])]

# Create a GeoDataFrame
transport_vertices = gpd.GeoDataFrame(transport_vertices, geometry=geometry)

In [ ]:
transport_vertices

### 4. Seperate vertices and edges and concatenate

You need to seperate the vertices and edges into vertex type streets and not streets as you only want to link the railway vertices to streets and not any of the created connectors, so you seperate and will rejoin again after running snkit

#### vertices

In [ ]:
vertices2 = vertices.loc[vertices['vertex_typ']=='street']

vertices3 = vertices.loc[vertices['vertex_typ']=='not_street']

In [ ]:
#concatenate the street vertices with the transport_vertices
vertices_network = pd.concat([vertices2, transport_vertices])

In [ ]:
vertices_network

In [ ]:
#reset_index
vertices_network = vertices_network.reset_index().drop('index', axis=1)

In [ ]:
vertices_network

#### Edges

In [ ]:
#rename
edges = edges.rename(columns = {'from_id':'source', 'to_id':'target' })

In [ ]:
edges

In [ ]:
#seperate street edges from connection edges
edges2 = edges.loc[edges['vertex_typ']!='not_street']
edges3 = edges.loc[edges['vertex_typ']=='not_street']

In [ ]:
edges2

In [ ]:
edges3

### 5. Snkit to link railway vertices with road network

- creating additional edges between streets and railway vertices

In [ ]:
#create a base network for snkit from the street edges and the street vertices and railways
base_network = snkit.Network(vertices_network, edges2) 

In [ ]:
# # Plot the edges and vertices
ax = plt.axes()
base_network.edges.plot(ax=ax, linewidth = 1, alpha = 0.6)
base_network.nodes.plot(ax=ax, facecolor = 'red', markersize = 3)

In [ ]:
base_network.nodes

In [ ]:
base_network.edges.head()

In [ ]:
#linking the stations to the network
linked = snkit.network.link_nodes_to_nearest_edge(base_network, condition=None)
#ignore warnings

In [ ]:
# plot
ax = plt.axes()
linked.edges.plot(ax=ax, linewidth = 1, alpha = 0.5)
linked.nodes.plot(ax=ax, facecolor = 'red', markersize = 3, alpha = 1)

In [ ]:
#Adding in the connector edges and vertices which we left out before

linked.nodes = pd.concat([linked.nodes, vertices3])

linked.edges = pd.concat([linked.edges, edges3])

In [ ]:
linked.nodes

In [ ]:
linked.edges

In [ ]:
linked.edges = linked.edges[['edge_type', 'vertex_typ','source', 'target', 'geometry']]

In [ ]:
linked.nodes = linked.nodes.rename(columns = {'id':'vertex_id'})

In [ ]:
linked.nodes['x'] = linked.nodes['geometry'].x # x coord of column
linked.nodes['y'] = linked.nodes['geometry'].y #y coord of column

In [ ]:
# create IDs for the new vertices and edges, it renames all the ids 
with_id = snkit.network.add_topology(snkit.network.add_ids(linked))

### 6. Adjusting attributes 

In [ ]:
with_id.edges.head()

In [ ]:
with_id.edges = with_id.edges[['edge_type', 'vertex_typ','source', 'target', 'id','from_id','to_id','geometry']]

In [ ]:
with_id.edges['edge_type'] = with_id.edges['edge_type'].fillna('brt_connector') #could be a different connector i.e BRT connector depending on mode

In [ ]:
with_id.edges['vertex_typ'] = with_id.edges['vertex_typ'].fillna('not_street')

In [ ]:
with_id.nodes

In [ ]:
with_id.nodes['vertex_typ'] = with_id.nodes['vertex_typ'].fillna('not_street')

In [ ]:
with_id.nodes['land use'] = with_id.nodes['land use'].fillna('brt_connector') #could be a different connector i.e BRT connector depending on mode

In [ ]:
#Subset the dataframes

In [ ]:
edges = with_id.edges

In [ ]:
vertices = with_id.nodes

create a dictionary of old vertex names of transport stops/station and new names, we do this so that the station names can replace the new vertex IDs, this is so that later on when the transportation edges are connected they are connected by the station names.

In [ ]:
#you need to include all the different kinds of stops/stations you currently have in the network each time you run this
subset = vertices[vertices["land use"].isin(["brt_stop", "railway_stop"])] 

In [ ]:
subset

In [ ]:
my_dict = dict(zip(subset['id'], subset['vertex_id']))

In [ ]:
my_dict

In [ ]:
#transform id vertices from the automated ones to the station/stop names

In [ ]:
vertices['id'] = vertices['id'].replace(my_dict)
edges['from_id'] = edges['from_id'].replace(my_dict)
edges['to_id'] = edges['to_id'].replace(my_dict)

In [ ]:
# get rid of source, target and vertex_id

In [ ]:
edges = edges.drop(['source', 'target'], axis =1)

In [ ]:
edges 

In [ ]:
vertices

In [ ]:
vertices = vertices.drop(['vertex_id'], axis =1)

### 7. Saving the files to shp

In [ ]:
path3 = ""

In [ ]:
os.chdir(path3)
for file in glob.glob("*"):
    print(file)

In [ ]:
edges.to_file('edges_b')

In [ ]:
vertices.to_file('vertices_b')